In [ ]:
import os, subprocess
import numpy as np

In [ ]:
# First we define some paths
cwd = os.getcwd()
datadir = os.path.join(cwd, "data")
masterdir = os.path.join(cwd, "Masters")
sciencedir = os.path.join(cwd, "Science")

To get started with `pypeit`, we run it on all the images we have (they are located in `data` and are in `fits.fz` format). `pypeit` reads those files and tries to determine which kind of exposure they are. Based on their properties (calibration image, science image, etc.), different 'setups' are created. Let's try it.

In the next cell, we run a commandline argument on our data directory. The -s command specifies the spectrograph, in our case ALFOSC, located at the Nordic Optical Telescope (NOT) on La Palma. Note: `subprocess` launches an external command from the python script. This is neccessary because `pypeit` is command-line based.

In [ ]:
subprocess.run(f"pypeit_setup -r {datadir} -s not_alfosc -c all", shell=True)

Now, look into the folder of this notebook. You will see that pypeit has created several new directories named after the spectrograph: `not_alfosc_A`, `not_alfosc_B` etc. You can delete `not_alfosc_C`, `not_alfosc_D` and `not_alfosc_E`, as we won't need them.

Now take a look at `not_alfosc_B/not_alfosc_B.pypeit`. This file contains all images belonging to a certain setup. Look at the `frametype`-column. Some lines are `arc`, some are `pixelflat, illumflat, tilt`, some are `science`.

### **Task 1** 
Look up and what the different frametypes are.

The data files just processed contain observations of three different supernovae (`ZTF22......`) and one standard star (`SP1045+378`). The latter is a very well measured star (usually, we have Hubble spectra of those stars). This so-called standard star is used to convert the measured signal of our supernova into physical flux. This form of calibration is called 'fluxing', we will do that later.

In the subsequent steps, we will be interested in our standard star `SP1045+378` and one of the three supernovae we have observed, namely `ZTF22aajhkpo`.

The procedure now will be as follows:
- Reduce the spectrum of the standard star
- Reduce the spectra of the supernova ZTF22aajhkpo (there are two individual exposures)
- Calibrate the two spectra of ZTF22aajhkpo with the standard star (fluxing)
- Combine the two spectra of ZTF22aajhkpo into a final calibrated spectrum (coadding)

So, let's start with reducing the standard star spectrum.

Sadly, `pypeit` did not correctly assign `bias` images to the setup. 

### **Task 2** 
Look up what bias images are and shortly describe what they are used for.

After that, we will skip a step of removing some of the lines from the `.pypeit` file (and adding some of the `not_alfosc_A`-setup and use a ready-made file provided by me. This file is located at `reduction_files/standard.pypeit`. Take a look at it and compare it to `not_alfosc_B/not_alfosc_B.pypeit`. You will see that there are bias-images from the A-setup added and some calibration files removed. Don't fret, the `reduction_files/standard.pypeit` file is what we will use from now.

In [ ]:
infile_standard = os.path.join("reduction_files", "standard.pypeit")
subprocess.run(f"run_pypeit {infile_standard}", shell=True)

This will have created a reduced spectrum of the standard star, located at `Science/spec1d_ALFe200084-SP1045+378_ALFOSC_20220521T002248.039.fits`. You can look at it by running the next cell:

In [ ]:
infile_standard_reduced = os.path.join(sciencedir, "spec1d_ALFe200084-SP1045+378_ALFOSC_20220521T002248.039.fits")
subprocess.run(f"pypeit_show_1dspec {infile_standard_reduced}", shell=True)

Ok cool, a very nice spectrum of a bright and well-behaved star 🎉. Now, let's reduce the supernova.

In [ ]:
infile_supernova = os.path.join("reduction_files", "ZTF22aajhkpo.pypeit")
subprocess.run(f"run_pypeit {infile_supernova}", shell=True)

After the reduction is done, you can look at some of the quality metrics in `QA/MF_B.html`

The first plot shows the quality of the wavelength calibration. For this, `pypeit` has stored information on which wavelength the spectroscope should detect when taking an image of one of the arc lamps. These are then fit. From the fit, a mapping of image pixels and wavelength can be derived.

As we now have three spectra - one of the standard star, and two of our supernova (look in the `Science` directory), we need to flux calibrate the supernova spectra with the standard star. 

To do so, we create a sensitivity function (encoding on how much atmospheric extinction there is per wavelength. As the standard star has a template image taken by Hubble in space, we can infer extinction from our freshly reduced standard star spectrum). To do so, we run:

In [ ]:
outfile_sensfunc = "sensfunc.fits"
subprocess.run(f"pypeit_sensfunc {infile_standard_reduced} -o {outfile_sensfunc}", shell=True)

All the calibration information is now stored in `sensfunc.fits`. You can look at the computed throughput by viewing `sensfunc_throughput.pdf`; as you can see, the throughput quite strongly depends on wavelength.

We need to apply the sensitivity function file (`sensfunc.fits`) to our two supernova spectra. To do so, there is a little instruction-textfile that I have already created and which is located at `reduction_files/fluxing.txt`. The actual command is:

In [ ]:
flux_file = os.path.join("reduction_files", "fluxing.txt")
subprocess.run(f"pypeit_flux_calib {flux_file}", shell=True)

This command overwrites the two supernova spectra already present in the `Science` folder.

Now we need to combine those two spectra to increase our signal-to-noise, a process called 'coadding'. To do so, again, a little helper file is needed which is located at `reduction_files/coadd.txt`. We finally run:

In [ ]:
coadd_file = os.path.join("reduction_files", "coadd.txt")
subprocess.run(f"pypeit_coadd_1dspec {coadd_file}", shell=True)

Now we have created a spectrum out of two (fluxed) exposures. This is great, now we can do science with it! To look at the spectrum, run:

In [ ]:
infile = "ZTF22aajhkpo_coadd.fits"
subprocess.run(f"lt_xspec {infile}", shell=True)

A nice spectrum that is nonetheless not usable for wavelengths below 4000 Å. We now export this from the final fits file to have something more practical to handle.

In [ ]:
from astropy.io import fits
from astropy.table import Table
import pandas

In [ ]:
infile = "ZTF22aajhkpo_coadd.fits"

In [ ]:
h = fits.open(infile)
d = h[1].data

tab = Table.read(infile, format='fits')
spectrum = tab.to_pandas().drop(columns=["wave_grid_mid", "ivar", "mask"])
spectrum

### **Task 3** 
Cut the spectrum (remove everything below 4000 Å and above 9500 Å) and plot it.

Now, we want to know *which type of supernova* this spectrum is. 

Head over to https://astrobites.org/2016/12/02/classifying-supernovae/ and look at the schema. There are three elements that are crucial for a first classification.

### **Task 4** 
Find the rest-frame wavelength of the most prominent emission lines of these three elements and classify the supernova. 

**Tip 1**: You will need to account for redshift. Play around with the redshift, until the element(s) neatly fit. Note: NOT is not extremely sensitive (it's only a 2.6 m telescope), so high redshift values (> 1.0) are very unplausible.

**Tip 2**: Some lines could be broadened (bonus question: why?)